In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=4b403c255623015165da846da9902deae14c09eebe23ecbfed1767da881a685a
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import os

import matplotlib.pyplot as plt


# 경고 끄기
warnings.filterwarnings(action='ignore')

## 입력 shape 및 형태 정의 함수

In [ ]:
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

## Transformer 정의
- encoder

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):

    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [ ]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

## keras eraly stop, chekpoint 정의

In [ ]:
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=100)

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/check2') == False:
        os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/check2')

    filename = f'/content/drive/MyDrive/농산물예측/aT_data/check2/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

## Model 훈련 함수

In [ ]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 1):


    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )


    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

## 시점 윈도우 생성 함수

In [ ]:
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

## 데이터 불러오기 및 parameter 설정

In [ ]:

data_list = glob('/content/drive/MyDrive/농산물예측/aT_data/data/train/*.csv')
epoch = 1000
batch = 15
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

In [ ]:
data_list

['/content/drive/MyDrive/농산물예측/aT_data/data/train/train_1.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_0.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_5.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_28.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_34.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_16.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_36.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_18.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_13.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_21.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_11.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_33.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_15.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_25.csv',
 '/content/drive/MyDrive/농산물예측/aT_data/data/train/train_7.csv',
 '/content/drive/MyDrive/농산물예

## Train 과정

In [ ]:
weather = pd.read_csv('/content/drive/MyDrive/농산물예측/final_weather_with_pummok.csv')


for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    # 날씨 변환한거 다시바꿔주고 넣어주기
    weather_df = weather[weather['품목']==int(df_number)]
    df['주산지_0_초기온도(℃)']  = weather_df['주산지0_초기온도']
    df['주산지_0_최대온도(℃)'] = weather_df['주산지0_최대온도']
    df['주산지_0_최저온도(℃)']= weather_df['주산지0_최저온도']
    df['주산지_0_평균온도(℃)']= weather_df['주산지0_평균온도']
    df['주산지_0_강수량(ml)']= weather_df['주산지0_강수량(ml)']

    df['주산지_1_초기온도(℃)']  = weather_df['주산지1_초기온도']
    df['주산지_1_최대온도(℃)'] = weather_df['주산지1_최대온도']
    df['주산지_1_최저온도(℃)']= weather_df['주산지1_최저온도']
    df['주산지_1_평균온도(℃)']= weather_df['주산지1_평균온도']
    df['주산지_1_강수량(ml)']= weather_df['주산지1_강수량(ml)']   

    df['주산지_2_초기온도(℃)']  = weather_df['주산지2_초기온도']
    df['주산지_2_최대온도(℃)'] = weather_df['주산지2_최대온도']
    df['주산지_2_최저온도(℃)']= weather_df['주산지2_최저온도']
    df['주산지_2_평균온도(℃)']= weather_df['주산지2_평균온도']
    df['주산지_2_강수량(ml)']= weather_df['주산지2_강수량(ml)']


    # 습도 날리고
    df.drop('주산지_0_습도(%)',axis=1,inplace=True)
    df.drop('주산지_1_습도(%)',axis=1,inplace=True)
    df.drop('주산지_2_습도(%)',axis=1,inplace=True)





    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)

    # nan 처리
    df = df.fillna(0)

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch)
    transformer.load_weights(f'/content/drive/MyDrive/농산물예측/aT_data/check2/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model2') == False:
        os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model2')

    # 모델 저장
    transformer.save(f'/content/drive/MyDrive/농산물예측/aT_data/model2/transformer-{df_number}-{epoch}-{batch}.h5')

Output hidden; open in https://colab.research.google.com to view.

## Test 과정

In [ ]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/data/test/set_{i}/*.csv')

    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)

        # 일단 평균값으로 확인해보자
        df['주산지_0_초기온도(℃)']  = df['주산지_0_초기온도(℃)'].fillna(df['주산지_0_초기온도(℃)'].mean())
        df['주산지_0_최대온도(℃)'] = df['주산지_0_최대온도(℃)'].fillna(df['주산지_0_최대온도(℃)'].mean())
        df['주산지_0_최저온도(℃)']= df['주산지_0_최저온도(℃)'].fillna(df['주산지_0_최저온도(℃)'].mean())
        df['주산지_0_평균온도(℃)']= df['주산지_0_평균온도(℃)'].fillna(df['주산지_0_평균온도(℃)'].mean())
        df['주산지_0_강수량(ml)']= df['주산지_0_강수량(ml)'].fillna(df['주산지_0_강수량(ml)'].mean())

        df['주산지_1_초기온도(℃)']  = df['주산지_1_초기온도(℃)'].fillna(df['주산지_1_초기온도(℃)'].mean())
        df['주산지_1_최대온도(℃)'] = df['주산지_1_최대온도(℃)'].fillna(df['주산지_1_최대온도(℃)'].mean())
        df['주산지_1_최저온도(℃)']= df['주산지_1_최저온도(℃)'].fillna(df['주산지_1_최저온도(℃)'].mean())
        df['주산지_1_평균온도(℃)']= df['주산지_1_평균온도(℃)'].fillna(df['주산지_1_평균온도(℃)'].mean())
        df['주산지_1_강수량(ml)']= df['주산지_1_강수량(ml)'].fillna(df['주산지_1_강수량(ml)'].mean()) 

        df['주산지_2_초기온도(℃)']  = df['주산지_2_초기온도(℃)'].fillna(df['주산지_2_초기온도(℃)'].mean())
        df['주산지_2_최대온도(℃)'] = df['주산지_2_최대온도(℃)'].fillna(df['주산지_2_최대온도(℃)'].mean())
        df['주산지_2_최저온도(℃)']= df['주산지_2_최저온도(℃)'].fillna(df['주산지_2_최저온도(℃)'].mean())
        df['주산지_2_평균온도(℃)']= df['주산지_2_평균온도(℃)'].fillna(df['주산지_2_평균온도(℃)'].mean())
        df['주산지_2_강수량(ml)']= df['주산지_2_강수량(ml)'].fillna(df['주산지_2_강수량(ml)'].mean())




        
        # 습도 날리고
        df.drop('주산지_0_습도(%)',axis=1,inplace=True)
        df.drop('주산지_1_습도(%)',axis=1,inplace=True)
        df.drop('주산지_2_습도(%)',axis=1,inplace=True)

        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)


        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))


        # model test
        if os.path.exists('/content/drive/MyDrive/농산물예측/aT_data/model_output2') == False:
            os.mkdir('/content/drive/MyDrive/농산물예측/aT_data/model_output2')

        if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model_output2/set_{i}') == False:
            os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model_output2/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'/content/drive/MyDrive/농산물예측/aT_data/model2/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'/content/drive/MyDrive/농산물예측/aT_data/model_output2/set_{i}/predict_{file_number}.csv', index=False)

100%|██████████| 10/10 [08:46<00:00, 52.69s/it]


In [ ]:
for k in tqdm(range(10)):

  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/model_output2/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:24<00:00,  2.41s/it]


In [ ]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.1250,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.2800,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,4939.5160,1023.865540,1671.473600,983.931640,26.892850,4434.554700,3396.8670,3214.258500,3585.693000,71.416300,...,5533.140600,67.526270,70.709700,66.242905,3671.027600,2109.659000,4171.336000,1409.132700,1312.860400,1414.169900
2,4400.2915,906.728800,1720.387500,847.096300,25.414658,4545.467000,3388.6443,2473.809000,3614.085700,71.085350,...,2981.330600,67.255000,71.408676,65.894295,3934.045000,2122.564200,4141.790000,-525.419560,1715.269400,1421.259800
3,2138.0942,-53.681328,705.838200,686.431460,25.828304,4561.154300,3385.8535,1388.676400,747.109900,70.370520,...,1407.971600,67.605995,71.215290,67.634740,305.321780,2157.068400,971.793800,-1188.328000,214.775090,1384.348800
4,5032.4710,968.172800,2492.112300,820.560900,26.530117,4533.899400,3409.5364,3369.622800,3688.495400,69.127266,...,5048.651400,68.438810,71.578064,67.237090,4843.087000,2095.170700,4286.249500,535.628500,1822.773100,1357.174300
5,4843.8480,943.524540,2140.743700,1241.564000,26.928898,4631.431000,3476.6165,3106.569000,4009.317000,71.151245,...,5101.137000,67.631035,70.158990,67.060480,4116.975000,2119.192900,4067.092000,-113.842735,2080.244000,1385.134800
6,4961.3640,967.721700,2155.910400,1119.945800,26.019728,4477.831500,3436.2012,3196.449700,3927.979200,71.558014,...,5128.952000,68.278320,71.391396,68.575440,4165.267000,2084.580300,3934.159000,672.049600,1838.982000,1405.766100
7,4960.1143,951.671400,2172.472400,1110.998500,25.043749,4728.229500,3501.9229,3297.031200,3915.867400,71.696040,...,5125.147500,67.288536,72.145874,67.716370,4478.501500,2076.087000,3438.606700,520.989700,2036.887600,1434.944000
8,4860.3040,987.872100,1529.187300,896.220640,25.989323,4591.619000,3430.8137,3048.628000,3389.559600,69.963770,...,4831.399400,68.403830,70.871870,67.577250,3522.695600,2082.733400,3832.780800,1358.147000,1761.579600,1428.076400
9,4483.6206,864.675900,1575.997400,776.951300,27.100710,4769.579000,3466.6475,2450.482200,3413.321000,68.273750,...,3660.946500,67.448640,71.406370,66.480130,3832.232200,2124.139400,3736.890400,71.983500,1945.132300,1414.530400


In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

In [ ]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('/content/drive/MyDrive/농산물예측/aT_data/answer_example2.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv('/content/drive/MyDrive/농산물예측/aT_data/submit3.csv',index=False)

In [ ]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
